In [2]:
# !pip install PyQt5
# ! git clone --recursive https://github.com/dmlc/xgboost
# !pip install pandastable
# !pip install -U numpy scipy py_entitymatching
!pip install javaobj-py3

     ---------------------------------------- 0.0/57.3 kB ? eta -:--:--
     --------------------------- ---------- 41.0/57.3 kB 653.6 kB/s eta 0:00:01
     -------------------------------------- 57.3/57.3 kB 747.3 kB/s eta 0:00:00


In [12]:
import javaobj

with open("data/data/cleanCleanErDatasets/acmProfiles", "rb") as fd:
    pobj = javaobj.v2.load(fd)

print(pobj[1].get_class().fields_names)
print(type(pobj))
print(type(pobj[1]), pobj[1])
pobj[1].attributes.load_from_instance()
print(type(pobj[1].attributes), pobj[10].attributes.get_class())

['attributes', 'entityUrl']
<class 'javaobj.v2.transformers.JavaList'>
<class 'javaobj.v2.beans.JavaInstance'> [instance 0x7e0016: type org.scify.jedai.datamodel.EntityProfile]
<class 'javaobj.v2.transformers.JavaSet'> [classdesc 0x7e0006: name java.util.HashSet, uid -5024744406713321676]


In [112]:
import javaobj
import csv
import pandas as pd

# Step 1: Read the serialized object file
with open('data/data/cleanCleanErDatasets/acmProfiles', 'rb') as f:
    data = javaobj.v2.load(f)

df = pd.DataFrame()
    
rows = []
for obj in data:
    row = []
    for key, value in obj.__dict__.items():
        if key == 'classdesc':
            field_names = [field.name for field in value.fields]
            field_values = []
            for field in value.fields:
                attr_value = getattr(obj, field.name)
                if isinstance(attr_value, set):
                    pd_row = dict()
                    for ins in set(attr_value):
                        class_desc = ins.classdesc
                        key = ''
                        for field_desc in class_desc.fields:   
                            attr_name = field_desc.name
                            if attr_name == 'name':
                                key = getattr(ins, attr_name)
                            if attr_name == 'value':
                                attr_value = getattr(ins, attr_name)
                                pd_row[key]=attr_value
                    pd_row = pd.DataFrame(pd_row, index=[0])
                    df = pd.concat([df, pd_row], ignore_index=True)
#                 else:
#                     field_values
            row.append(field_values)
        else:
            row.append(value)
    rows.append(row)
with open('data.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(rows)

In [113]:
df

,authors,title,venue,year
0,"Gottfried Vossen, Mathias Weske",The WASA2 object-oriented workflow management ...,International Conference on Management of Data,1999
1,"Isabel F. Cruz, Kimberly M. James",A user-centered interface for querying distrib...,International Conference on Management of Data,1999
2,"Athman Bouguettaya, Boualem Benatallah, Lily H...","World Wide Database-integrating the Web, CORBA...",International Conference on Management of Data,1999
3,"Chaitan Baru, Amarnath Gupta, Bertram Lud&#228...",XML-based information mediation with MIX,International Conference on Management of Data,1999
4,"Alexander Brodsky, Victor E. Segal, Jia Chen, ...",The CCUBE constraint object-oriented database ...,International Conference on Management of Data,1999
...,...,...,...,...
2289,"Alfons Kemper, Donald Kossmann",Dual-Buffering Strategies in Object Bases,Very Large Data Bases,1994
2290,"Philip A. Bernstein, Yannis Ioannidis, Raghu R...",Guest editorial,The VLDB Journal &mdash; The International Jou...,2003
2291,Ralf Hartmut G&#252;ting,GraphDB: Modeling and Querying Graphs in Datab...,Very Large Data Bases,1994
2292,Alexander A. Anisimov,Review of The data warehouse toolkit: the comp...,ACM SIGMOD Record,2003


In [6]:
import javaobj
import csv

# Step 1: Read the serialized object file
with open('data/data/cleanCleanErDatasets/acmProfiles', 'rb') as f:
    data = javaobj.load(f)

# Step 2: Convert the list of instances to a list of lists
rows = []
for obj in data:
    row = []
    for attr_name in dir(obj):
        if not callable(getattr(obj, attr_name)) and not attr_name.startswith('__'):
            row.append(str(getattr(obj, attr_name)))
    rows.append(row)
print(row)
# # Step 3: Write the data to a CSV file
# with open('data.csv', 'w', newline='') as f:
#     writer = csv.writer(f)
#     writer.writerows(rows)

['[]', 'JavaSet({<javaobj:org.scify.jedai.datamodel.Attribute>, <javaobj:org.scify.jedai.datamodel.Attribute>, <javaobj:org.scify.jedai.datamodel.Attribute>, <javaobj:org.scify.jedai.datamodel.Attribute>})', '[org.scify.jedai.datamodel.EntityProfile:0x1B2B0CD02BE6E37]', '672979']


In [90]:
class JavaMap(dict, javaobj.v2.beans.JavaInstance):
    """
    Inherits from dict for Python usage, JavaInstance for parsing purpose
    """
    def __init__(self):
        # Don't forget to call both constructors
        dict.__init__(self)
        javaobj.v2.beans.JavaInstance.__init__(self)

    def load_from_blockdata(self, parser, reader, indent=0):
#     """
#     Reads content stored in a block data.

#     This method is called only if the class description has both the
#     `SC_EXTERNALIZABLE` and `SC_BLOCK_DATA` flags set.

#     The stream parsing will stop and fail if this method returns False.

#     :param parser: The JavaStreamParser in use
#     :param reader: The underlying data stream reader
#     :param indent: Indentation to use in logs
#     :return: True on success, False on error
#     """
    # This kind of class is not supposed to have the SC_BLOCK_DATA flag set
        return False

    def load_from_instance(self, indent=0):
        # type: (int) -> bool
        """
        Load content from the parsed instance object.

        This method is called after the block data (if any), the fields and
        the annotations have been loaded.

        :param indent: Indentation to use while logging
        :return: True on success (currently ignored)
        """
        # Maps have their content in their annotations
        for cd, annotations in self.annotations.items():
            # Annotations are associated to their definition class
            if cd.name == "java.util.HashMap":
                # We are in the annotation created by the handled class
                # Group annotation elements 2 by 2
                # (storage is: key, value, key, value, ...)
                args = [iter(annotations[1:])] * 2
                for key, value in zip(*args):
                    self[key] = value

                # Job done
                return True

        # Couldn't load the data
        return False

class MapObjectTransformer(javaobj.v2.api.ObjectTransformer):
    """
    Creates a JavaInstance object with custom loading methods for the
    classes it can handle
    """
    def create_instance(self, classdesc):
        # type: (JavaClassDesc) -> Optional[JavaInstance]
        """
        Transforms a parsed Java object into a Python object

        :param classdesc: The description of a Java class
        :return: The Python form of the object, or the original JavaObject
        """
        if classdesc.name == "java.util.HashMap":
            # We can handle this class description
            return JavaMap()
        else:
            # Return None if the class is not handled
            return None

In [44]:
class BaseTransformer(javaobj.v2.transformers.ObjectTransformer):
    """
    Creates a JavaInstance object with custom loading methods for the
    classes it can handle
    """

    def __init__(self, handled_classes=None):
        self.instance = None
        self.handled_classes = handled_classes or {}

    def create_instance(self, classdesc):
        """
        Transforms a parsed Java object into a Python object

        :param classdesc: The description of a Java class
        :return: The Python form of the object, or the original JavaObject
        """
        if classdesc.name in self.handled_classes:
            self.instance = self.handled_classes[classdesc.name]()
            return self.instance

        return None

class RandomChildTransformer(BaseTransformer):
    def __init__(self):
        super(RandomChildTransformer, self).__init__(
            {"RandomChild": RandomChildInstance}
        )

class CustomWriterTransformer(BaseTransformer):
    def __init__(self):
        super(CustomWriterTransformer, self).__init__(
            {"CustomWriter": CustomWriterInstance}
        )

class JavaRandomTransformer(BaseTransformer):
    def __init__(self):
        super(JavaRandomTransformer, self).__init__()
        self.name = "java.util.Random"
        self.field_names = ["haveNextNextGaussian", "nextNextGaussian", "seed"]
        self.field_types = [
            javaobj.v2.beans.FieldType.BOOLEAN,
            javaobj.v2.beans.FieldType.DOUBLE,
            javaobj.v2.beans.FieldType.LONG,
        ]

    def load_custom_writeObject(self, parser, reader, name):
        if name != self.name:
            return None

        fields = []
        values = []
        for f_name, f_type in zip(self.field_names, self.field_types):
            values.append(parser._read_field_value(f_type))
            fields.append(javaobj.beans.JavaField(f_type, f_name))

        class_desc = javaobj.beans.JavaClassDesc(
            javaobj.beans.ClassDescType.NORMALCLASS
        )
        class_desc.name = self.name
        class_desc.desc_flags = javaobj.beans.ClassDataType.EXTERNAL_CONTENTS
        class_desc.fields = fields
        class_desc.field_data = values
        return class_desc

In [45]:
class CustomWriterInstance(javaobj.v2.beans.JavaInstance):
    def __init__(self):
        javaobj.v2.beans.JavaInstance.__init__(self)

    def load_from_instance(self):
        """
        Updates the content of this instance
        from its parsed fields and annotations
        :return: True on success, False on error
        """
        if self.classdesc and self.classdesc in self.annotations:
            # Here, we known there is something written before the fields,
            # even if it's not declared in the class description
            fields = ["int_not_in_fields"] + self.classdesc.fields_names
            raw_data = self.annotations[self.classdesc]
            int_not_in_fields = struct.unpack(
                ">i", BytesIO(raw_data[0].data).read(4)
            )[0]
            custom_obj = raw_data[1]
            values = [int_not_in_fields, custom_obj]
            self.field_data = dict(zip(fields, values))
            return True

        return False


class RandomChildInstance(javaobj.v2.beans.JavaInstance):
    def load_from_instance(self):
        """
        Updates the content of this instance
        from its parsed fields and annotations
        :return: True on success, False on error
        """
        if self.classdesc and self.classdesc in self.field_data:
            fields = self.classdesc.fields_names
            values = [
                self.field_data[self.classdesc][self.classdesc.fields[i]]
                for i in range(len(fields))
            ]
            self.field_data = dict(zip(fields, values))
            if (
                self.classdesc.super_class
                and self.classdesc.super_class in self.annotations
            ):
                super_class = self.annotations[self.classdesc.super_class][0]
                self.annotations = dict(
                    zip(super_class.fields_names, super_class.field_data)
                )
            return True

        return False

In [53]:
# Load the object using those transformers
transformers = [
    CustomWriterTransformer(),
    RandomChildTransformer(),
    JavaRandomTransformer()
]
with open("data/data/cleanCleanErDatasets/abtBuyIdDuplicates", "rb") as fd:
    pobj = javaobj.v2.load(fd)

# Here we show a field that isn't visible from the class description
# The field belongs to the class but it's not serialized by default because
# it's static. See: https://stackoverflow.com/a/16477421/12621168
print(pobj.get_class().fields_names)

[]
